In [1]:
!pip install -r requirements.txt

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
import MySQLdb
import numpy as np
import pandas as pd

from sqlalchemy import create_engine

In [4]:
connection_string = 'mysql://{}:{}@{}/{}'.format(os.getenv('DB_USERNAME'), os.getenv('DB_PASSWORD'), \
                                                 os.getenv('DB_HOST'), os.getenv('DB_DATABASE'))
engine = create_engine(connection_string, echo=True)

In [5]:
sql_query = open('../data/sql/devices.sql').read()

In [6]:
df = pd.read_sql(sql_query, engine)

2020-07-30 17:32:26,356 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2020-07-30 17:32:26,359 INFO sqlalchemy.engine.base.Engine ()
2020-07-30 17:32:26,383 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2020-07-30 17:32:26,384 INFO sqlalchemy.engine.base.Engine ()
2020-07-30 17:32:26,428 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2020-07-30 17:32:26,429 INFO sqlalchemy.engine.base.Engine ()
2020-07-30 17:32:26,468 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8mb4' and `Collation` = 'utf8mb4_bin'
2020-07-30 17:32:26,469 INFO sqlalchemy.engine.base.Engine ()
2020-07-30 17:32:26,492 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2020-07-30 17:32:26,493 INFO sqlalchemy.engine.base.Engine ()
2020-07-30 17:32:26,514 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2020-07-30 17:32:26,515 INFO sqlalchemy.engine.base.E

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306469 entries, 0 to 306468
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            306469 non-null  int64 
 1   model         306469 non-null  object
 2   manufacturer  306469 non-null  object
 3   brand         306469 non-null  object
 4   os_version    306469 non-null  object
 5   is_root       306469 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 14.0+ MB


In [8]:
df.describe(include=['O'])

,model,manufacturer,brand,os_version
count,306469,306469,306469,306469
unique,15534,2817,3109,74
top,Redmi 5A,samsung,samsung,8.1.0
freq,3270,84849,83911,46003


In [9]:
mappings = {'id':'uint32', 'model':'category', 'manufacturer':'category',
            'brand':'category', 'os_version':'category', 'is_root': 'uint8'}

In [10]:
df = df.astype(mappings)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306469 entries, 0 to 306468
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   id            306469 non-null  uint32  
 1   model         306469 non-null  category
 2   manufacturer  306469 non-null  category
 3   brand         306469 non-null  category
 4   os_version    306469 non-null  category
 5   is_root       306469 non-null  uint8   
dtypes: category(4), uint32(1), uint8(1)
memory usage: 4.5 MB


In [11]:
df.to_parquet('../data/df.snappy.parquet', compression='snappy')
df.to_parquet('../data/df.brotli.parquet', compression='brotli')

In [12]:
%timeit pd.read_parquet('../data/df.snappy.parquet')

45.7 ms ± 465 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%timeit pd.read_parquet('../data/df.brotli.parquet')

61.9 ms ± 593 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
%timeit pq.read_table('../data/df.snappy.parquet').to_pandas()

45 ms ± 440 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
%timeit pq.read_table('../data/df.brotli.parquet').to_pandas()

61.2 ms ± 524 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
table = pa.Table.from_pandas(df)

In [24]:
%timeit pq.write_table(table, '../data/df.snappy.parquet', compression='SNAPPY')

95 ms ± 4.22 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
%timeit pq.write_table(table, '../data/df.brotli.parquet', compression='BROTLI')

471 ms ± 8.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%timeit pq.write_table(table, '../data/df.zstd.parquet', compression='ZSTD')

104 ms ± 3.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
%timeit df.to_csv('../data/df.csv', index=False)

1.28 s ± 15.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
